In [1]:
# %% [markdown]
# # Analiză Time Series - Energie Cinetică Pentilfuran cu FFT
# 
# Acest notebook analizează datele de energie cinetică pentru molecula de pentilfuran într-un câmp electric.
# Folosim transformata Fourier pentru a exploata natura sinusoidală a datelor.
# 
# **Parametri cunoscuți:**
# - Frecvența dominantă: 0.020000 Hz
# - Perioada principală: 50.00 pași de timp
# - Window size recomandat: 100 pași

# %% [markdown]
# ## 1. Import Libraries și Configurare

# %%
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization, Bidirectional
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import plotly.graph_objects as go
import plotly.subplots as sp
from scipy import signal
from scipy.fft import fft, fftfreq, ifft
import warnings
warnings.filterwarnings('ignore')

# Configurare pentru reproducibilitate
np.random.seed(42)
tf.random.set_seed(42)

print("📦 Libraries importate cu succes!")

# %% [markdown]
# ## 2. Încărcarea și Preprocesarea Datelor

# %%
# Încărcare date
my_file = "./pentilfuran.MDE"

df = pd.read_csv(
    my_file,
    sep=r"\s+",
    comment='#',
    names=["Step", "T", "E_KS", "E_tot", "Vol", "P"]
)

print(f"📊 Dimensiune date originale: {df.shape}")
print(f"📋 Coloane disponibile: {df.columns.tolist()}")
print(f"🔢 Numărul de steps unici: {df['Step'].nunique()}")

# %%
# Selectare liniile 1:901 pentru fiecare Step (optimizat)
df_data = (
    df.groupby("Step", group_keys=False)
    .apply(lambda g: g.iloc[1:901])
    .reset_index(drop=True)
)

print(f"✅ Dimensiunea după filtrare: {len(df_data)} rânduri")
print(f"📈 Range energie totală: [{df_data['E_tot'].min():.6f}, {df_data['E_tot'].max():.6f}]")
print(f"🌡️ Range temperatură: [{df_data['T'].min():.4f}, {df_data['T'].max():.4f}]")

# Verificare pentru valori lipsă
print(f"\n🔍 Valori lipsă per coloană:")
print(df_data.isnull().sum())

# %% [markdown]
# ## 3. Analiza Periodicității cu FFT

# %%
def analyze_periodicity(signal_data, sampling_rate=1.0, plot=True):
    """Analiză avansată de periodicitate cu FFT"""
    print("🔍 Analiză periodicitate cu FFT...")
    
    # Remove trend pentru FFT mai precisă
    detrended_signal = signal.detrend(signal_data)
    
    # FFT
    fft_values = fft(detrended_signal)
    frequencies = fftfreq(len(signal_data), d=1/sampling_rate)
    
    # Power spectrum (doar frecvențele pozitive, fără DC)
    power_spectrum = np.abs(fft_values[1:len(signal_data)//2])
    freqs_positive = frequencies[1:len(signal_data)//2]
    
    # Top 10 frecvențe dominante
    dominant_indices = np.argsort(power_spectrum)[-10:][::-1]
    dominant_freqs = freqs_positive[dominant_indices]
    dominant_powers = power_spectrum[dominant_indices]
    
    print(f"🎯 Top 10 frecvențe dominante:")
    for i, (freq, power) in enumerate(zip(dominant_freqs, dominant_powers)):
        period = 1/freq if freq != 0 else np.inf
        print(f"   {i+1:2d}. Freq: {freq:.6f} Hz, Perioadă: {period:8.2f} pași, Putere: {power:.2e}")
    
    if plot:
        # Plot FFT
        fig = sp.make_subplots(
            rows=2, cols=1,
            subplot_titles=['Semnal Original vs Detrended', 'Power Spectrum (FFT)']
        )
        
        # Semnal original vs detrended (primele 1000 puncte pentru vizibilitate)
        sample_size = min(1000, len(signal_data))
        x_axis = np.arange(sample_size)
        
        fig.add_trace(
            go.Scatter(x=x_axis, y=signal_data[:sample_size], 
                      name='Original', line=dict(color='blue')),
            row=1, col=1
        )
        fig.add_trace(
            go.Scatter(x=x_axis, y=detrended_signal[:sample_size], 
                      name='Detrended', line=dict(color='red')),
            row=1, col=1
        )
        
        # Power spectrum (zoom pe primele 50 frecvențe pentru claritate)
        freq_limit = min(50, len(freqs_positive))
        fig.add_trace(
            go.Scatter(x=freqs_positive[:freq_limit], y=power_spectrum[:freq_limit],
                      mode='lines+markers', name='Power Spectrum'),
            row=2, col=1
        )
        
        # Evidențiază frecvențele dominante
        for i, (freq, power) in enumerate(zip(dominant_freqs[:5], dominant_powers[:5])):
            if freq <= freqs_positive[freq_limit-1]:  # Doar dacă e în range-ul vizualizat
                fig.add_vline(x=freq, line_dash="dash", 
                             annotation_text=f"f{i+1}={freq:.4f}Hz", 
                             row=2, col=1)
        
        fig.update_layout(height=800, title_text="Analiză FFT - Energie Totală")
        fig.update_xaxes(title_text="Timp (pași)", row=1, col=1)
        fig.update_xaxes(title_text="Frecvența (Hz)", row=2, col=1)
        fig.update_yaxes(title_text="Energie", row=1, col=1)
        fig.update_yaxes(title_text="Amplitudine", row=2, col=1)
        fig.show()
    
    return dominant_freqs, power_spectrum, frequencies, detrended_signal

# Rulare analiză FFT pe energia totală
energy_data = df_data['E_tot'].values
dominant_freqs, power_spectrum, frequencies, detrended_energy = analyze_periodicity(energy_data)

# %% [markdown]
# ## 4. Crearea Features Fourier

# %%
def create_fourier_features(data, dominant_freqs, top_n=5):
    """Creează features bazate pe componentele Fourier dominante"""
    print(f"🌊 Creare {top_n} features Fourier...")
    
    fourier_features = []
    feature_names = []
    
    t = np.arange(len(data))
    
    for i, freq in enumerate(dominant_freqs[:top_n]):
        # Sin și Cos pentru fiecare frecvență dominantă
        sin_component = np.sin(2 * np.pi * freq * t)
        cos_component = np.cos(2 * np.pi * freq * t)
        
        fourier_features.extend([sin_component, cos_component])
        feature_names.extend([f'sin_f{i+1}_{freq:.4f}Hz', f'cos_f{i+1}_{freq:.4f}Hz'])
    
    print(f"✅ Features Fourier create: {feature_names}")
    return np.array(fourier_features).T, feature_names

# Crearea features Fourier
fourier_features, fourier_names = create_fourier_features(energy_data, dominant_freqs, top_n=3)
print(f"📏 Shape features Fourier: {fourier_features.shape}")

# %% [markdown]
# ## 5. Feature Engineering Complet

# %%
def create_comprehensive_features(df_data, fourier_features, fourier_names):
    """Creează un set complet de features pentru training"""
    print("🔧 Creare features comprehensive...")
    
    # Scalere pentru diferite tipuri de date
    energy_scaler = StandardScaler()
    temp_scaler = StandardScaler()
    pressure_scaler = StandardScaler()
    
    # Features de bază scalate
    energy_scaled = energy_scaler.fit_transform(df_data[['E_tot']]).flatten()
    temp_scaled = temp_scaler.fit_transform(df_data[['T']]).flatten()
    pressure_scaled = pressure_scaler.fit_transform(df_data[['P']]).flatten()
    
    # Features derivate pentru energie
    energy_diff = np.gradient(energy_scaled)
    energy_diff2 = np.gradient(energy_diff)  # Accelerație
    
    # Moving averages (pentru capturarea trend-urilor)
    energy_ma5 = pd.Series(energy_scaled).rolling(window=5, center=True).mean().fillna(method='bfill').fillna(method='ffill')
    energy_ma10 = pd.Series(energy_scaled).rolling(window=10, center=True).mean().fillna(method='bfill').fillna(method='ffill')
    energy_ma20 = pd.Series(energy_scaled).rolling(window=20, center=True).mean().fillna(method='bfill').fillna(method='ffill')
    
    # Volatilitate (rolling std)
    energy_vol5 = pd.Series(energy_scaled).rolling(window=5, center=True).std().fillna(0)
    energy_vol10 = pd.Series(energy_scaled).rolling(window=10, center=True).std().fillna(0)
    
    # Features pentru temperatură
    temp_diff = np.gradient(temp_scaled)
    temp_ma10 = pd.Series(temp_scaled).rolling(window=10, center=True).mean().fillna(method='bfill').fillna(method='ffill')
    
    # Combinare toate features
    all_features = np.column_stack([
        energy_scaled,              # Target principal
        temp_scaled,                # Temperatură
        pressure_scaled,            # Presiune
        energy_diff,                # Viteza energiei
        energy_diff2,               # Accelerația energiei
        energy_ma5,                 # Trend pe termen scurt
        energy_ma10,                # Trend pe termen mediu
        energy_ma20,                # Trend pe termen lung
        energy_vol5,                # Volatilitate scurtă
        energy_vol10,               # Volatilitate medie
        temp_diff,                  # Rata schimbării temperaturii
        temp_ma10,                  # Trend temperatură
        fourier_features           # Features Fourier
    ])
    
    # Namen features
    feature_names = [
        'Energy_scaled', 'Temp_scaled', 'Pressure_scaled',
        'Energy_velocity', 'Energy_acceleration',
        'Energy_MA5', 'Energy_MA10', 'Energy_MA20',
        'Energy_Vol5', 'Energy_Vol10',
        'Temp_velocity', 'Temp_MA10'
    ] + fourier_names
    
    print(f"✅ Total features: {all_features.shape[1]}")
    print(f"📋 Feature names: {feature_names}")
    
    return all_features, feature_names, {
        'energy_scaler': energy_scaler,
        'temp_scaler': temp_scaler,
        'pressure_scaler': pressure_scaler
    }

# Creare features complete
all_features, feature_names, scalers = create_comprehensive_features(df_data, fourier_features, fourier_names)

# Verificare pentru NaN sau Inf
print(f"\n🔍 Verificare calitatea features:")
print(f"NaN values: {np.isnan(all_features).sum()}")
print(f"Inf values: {np.isinf(all_features).sum()}")
print(f"Feature range: [{all_features.min():.4f}, {all_features.max():.4f}]")

# %% [markdown]
# ## 6. Crearea Secvențelor pentru LSTM

# %%
def create_sequences_optimized(features, energy_target, sequence_length, out_steps, overlap_ratio=0.8):
    """
    Creează secvențe optimizate cu overlap pentru mai multe date de training
    
    Args:
        features: Array cu toate features (include target la coloana 0)
        energy_target: Target-ul pentru predicție (energia scalată)
        sequence_length: Lungimea secvenței input
        out_steps: Numărul de pași de prezis
        overlap_ratio: Raportul de overlap între secvențe (0.8 = 80% overlap)
    """
    print(f"🔧 Creare secvențe cu parametri:")
    print(f"   - Sequence length: {sequence_length}")
    print(f"   - Output steps: {out_steps}")
    print(f"   - Overlap ratio: {overlap_ratio}")
    
    sequences, targets = [], []
    
    # Calculează pas-ul bazat pe overlap
    step_size = max(1, int(sequence_length * (1 - overlap_ratio)))
    
    # Generează secvențe cu overlap
    for i in range(0, len(features) - sequence_length - out_steps + 1, step_size):
        # Input sequence (toate features)
        seq = features[i:i + sequence_length]
        
        # Target sequence (doar energia)
        target = energy_target[i + sequence_length:i + sequence_length + out_steps]
        
        if len(target) == out_steps:  # Verifică că target-ul e complet
            sequences.append(seq)
            targets.append(target)
    
    sequences = np.array(sequences)
    targets = np.array(targets)
    
    print(f"✅ Secvențe create:")
    print(f"   - Input shape: {sequences.shape}")
    print(f"   - Target shape: {targets.shape}")
    print(f"   - Total samples: {len(sequences)}")
    
    return sequences, targets

# Parametrii optimizați pentru perioada de 50 pași
SEQUENCE_LENGTH = 100  # 2x perioada principală
OUT_STEPS = 25         # 0.5x perioada pentru predicții precise
OVERLAP_RATIO = 0.7    # 70% overlap pentru mai multe sample-uri

# Crearea secvențelor
energy_target = all_features[:, 0]  # Prima coloană e energia scalată
sequences, targets = create_sequences_optimized(
    all_features, 
    energy_target, 
    SEQUENCE_LENGTH, 
    OUT_STEPS,
    OVERLAP_RATIO
)

# %% [markdown]
# ## 7. Construirea Modelului LSTM Avansat

# %%
def build_advanced_lstm_model(input_shape, out_steps, dropout_rate=0.3):
    """
    Construiește model LSTM avansat optimizat pentru date periodice
    """
    print(f"🏗️ Construire model LSTM pentru:")
    print(f"   - Input shape: {input_shape}")
    print(f"   - Output steps: {out_steps}")
    print(f"   - Dropout rate: {dropout_rate}")
    
    model = Sequential([
        # Layer 1: Bidirectional LSTM pentru capturarea dependințelor în ambele direcții
        Bidirectional(
            LSTM(128, return_sequences=True, dropout=dropout_rate, recurrent_dropout=dropout_rate),
            input_shape=input_shape,
            name='bidirectional_lstm_1'
        ),
        BatchNormalization(name='batch_norm_1'),
        
        # Layer 2: Al doilea LSTM bidirectional
        Bidirectional(
            LSTM(64, return_sequences=True, dropout=dropout_rate, recurrent_dropout=dropout_rate),
            name='bidirectional_lstm_2'
        ),
        BatchNormalization(name='batch_norm_2'),
        
        # Layer 3: LSTM final
        LSTM(32, dropout=dropout_rate, recurrent_dropout=dropout_rate, name='lstm_final'),
        BatchNormalization(name='batch_norm_3'),
        
        # Dense layers cu regularizare
        Dense(64, activation='relu', name='dense_1'),
        Dropout(dropout_rate + 0.1, name='dropout_1'),
        
        Dense(32, activation='relu', name='dense_2'),
        Dropout(dropout_rate, name='dropout_2'),
        
        Dense(16, activation='relu', name='dense_3'),
        Dropout(dropout_rate * 0.5, name='dropout_3'),
        
        # Output layer
        Dense(out_steps, activation='linear', name='output')
    ])
    
    # Optimizer cu parametri optimizați
    optimizer = Adam(
        learning_rate=0.001,
        beta_1=0.9,
        beta_2=0.999,
        epsilon=1e-7
    )
    
    # Compilare cu loss function robust
    model.compile(
        optimizer=optimizer,
        loss='huber',  # Mai robust la outliers decât MSE
        metrics=['mae', 'mse', 'mape']
    )
    
    print(f"✅ Model construit cu succes!")
    print(f"📊 Parametri totali: {model.count_params():,}")
    
    return model

# Construire model
model = build_advanced_lstm_model(
    input_shape=(sequences.shape[1], sequences.shape[2]),
    out_steps=OUT_STEPS,
    dropout_rate=0.3
)

# Afișare arhitectura modelului
model.summary()

# %% [markdown]
# ## 8. Împărțirea Datelor și Antrenarea

# %%
# Split date (fără shuffle pentru time series)
X_train, X_test, y_train, y_test = train_test_split(
    sequences, targets, 
    test_size=0.2, 
    random_state=42, 
    shuffle=False  # Important pentru time series
)

print(f"📊 Împărțirea datelor:")
print(f"   - Train samples: {X_train.shape[0]}")
print(f"   - Test samples: {X_test.shape[0]}")
print(f"   - Features per sample: {X_train.shape[2]}")
print(f"   - Sequence length: {X_train.shape[1]}")

# Definire callbacks avansate
callbacks = [
    EarlyStopping(
        monitor='val_loss',
        patience=20,
        restore_best_weights=True,
        verbose=1,
        min_delta=1e-6
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=10,
        min_lr=1e-7,
        verbose=1,
        cooldown=5
    ),
    ModelCheckpoint(
        'best_energy_lstm_model.h5',
        monitor='val_loss',
        save_best_only=True,
        verbose=1,
        save_weights_only=False
    )
]

print("🚀 Începe antrenarea...")

# %% 
# Antrenare model
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=100,
    batch_size=32,
    callbacks=callbacks,
    verbose=1
)

print("✅ Antrenarea completată!")

# %% [markdown]
# ## 9. Evaluarea Modelului

# %%
def evaluate_comprehensive(model, X_test, y_test, scalers, feature_names):
    """Evaluare comprehensivă a modelului"""
    print("📈 Evaluare model...")
    
    # Predicții
    y_pred = model.predict(X_test, verbose=0)
    
    # Scalare inversă pentru metrici în unități originale
    energy_scaler = scalers['energy_scaler']
    
    # Flatten pentru calcularea metricilor
    y_test_flat = y_test.flatten().reshape(-1, 1)
    y_pred_flat = y_pred.flatten().reshape(-1, 1)
    
    # Scalare inversă
    y_test_original = energy_scaler.inverse_transform(y_test_flat).flatten()
    y_pred_original = energy_scaler.inverse_transform(y_pred_flat).flatten()
    
    # Calculare metrici
    mse = mean_squared_error(y_test_original, y_pred_original)
    mae = mean_absolute_error(y_test_original, y_pred_original)
    r2 = r2_score(y_test_original, y_pred_original)
    rmse = np.sqrt(mse)
    
    # MAPE (Mean Absolute Percentage Error)
    mape = np.mean(np.abs((y_test_original - y_pred_original) / y_test_original)) * 100
    
    print(f"🏆 Performanța modelului:")
    print(f"   ├── MSE: {mse:.8f}")
    print(f"   ├── MAE: {mae:.8f}")
    print(f"   ├── RMSE: {rmse:.8f}")
    print(f"   ├── R²: {r2:.6f}")
    print(f"   └── MAPE: {mape:.4f}%")
    
    # Analiza reziduurilor
    residuals = y_pred_original - y_test_original
    print(f"\n📊 Analiza reziduurilor:")
    print(f"   ├── Mean residual: {np.mean(residuals):.8f}")
    print(f"   ├── Std residual: {np.std(residuals):.8f}")
    print(f"   ├── Min residual: {np.min(residuals):.8f}")
    print(f"   └── Max residual: {np.max(residuals):.8f}")
    
    return {
        'predictions': y_pred,
        'predictions_original': y_pred_original,
        'targets_original': y_test_original,
        'residuals': residuals,
        'metrics': {
            'mse': mse,
            'mae': mae,
            'rmse': rmse,
            'r2': r2,
            'mape': mape
        }
    }

# Evaluare model
results = evaluate_comprehensive(model, X_test, y_test, scalers, feature_names)

# %% [markdown]
# ## 10. Vizualizări Complete

# %%
def create_comprehensive_plots(results, history):
    """Creează vizualizări complete pentru analiza modelului"""
    print("📊 Creare vizualizări...")
    
    # Extrage rezultatele
    y_pred_orig = results['predictions_original']
    y_test_orig = results['targets_original']
    residuals = results['residuals']
    metrics = results['metrics']
    
    # 1. Plot principal cu 4 subplots
    fig = sp.make_subplots(
        rows=2, cols=2,
        subplot_titles=[
            'Predicții vs Realitate', 
            'Training History', 
            'Distribuție Reziduuri', 
            'Time Series Comparison'
        ],
        specs=[[{"secondary_y": False}, {"secondary_y": False}],
               [{"secondary_y": False}, {"secondary_y": False}]]
    )
    
    # 1a. Scatter plot predicții vs realitate
    sample_size = min(2000, len(y_test_orig))
    indices = np.random.choice(len(y_test_orig), sample_size, replace=False)
    y_test_sample = y_test_orig[indices]
    y_pred_sample = y_pred_orig[indices]
    
    fig.add_trace(
        go.Scatter(
            x=y_test_sample, 
            y=y_pred_sample, 
            mode='markers', 
            name=f'Predicții (R²={metrics["r2"]:.4f})',
            marker=dict(size=4, opacity=0.6, color='blue'),
            hovertemplate='Real: %{x:.6f}<br>Pred: %{y:.6f}<extra></extra>'
        ),
        row=1, col=1
    )
    
    # Linia perfectă
    min_val, max_val = min(y_test_sample.min(), y_pred_sample.min()), max(y_test_sample.max(), y_pred_sample.max())
    fig.add_trace(
        go.Scatter(
            x=[min_val, max_val], 
            y=[min_val, max_val],
            mode='lines', 
            name='Perfect Fit', 
            line=dict(dash='dash', color='red')
        ),
        row=1, col=1
    )
    
    # 1b. Training history
    fig.add_trace(
        go.Scatter(y=history.history['loss'], name='Train Loss', line=dict(color='blue')),
        row=1, col=2
    )
    fig.add_trace(
        go.Scatter(y=history.history['val_loss'], name='Val Loss', line=dict(color='red')),
        row=1, col=2
    )
    
    # 1c. Histogramă reziduuri
    fig.add_trace(
        go.Histogram(
            x=residuals[indices], 
            name='Reziduuri', 
            nbinsx=50,
            marker=dict(color='green', opacity=0.7)
        ),
        row=2, col=1
    )
    
    # 1d. Time series comparison (primele 500 puncte)
    time_sample = min(500, len(y_test_orig))
    x_time = np.arange(time_sample)
    
    fig.add_trace(
        go.Scatter(x=x_time, y=y_test_orig[:time_sample], name='Original', 
                  line=dict(color='blue', width=2)),
        row=2, col=2
    )
    fig.add_trace(
        go.Scatter(x=x_time, y=y_pred_orig[:time_sample], name='Predicție', 
                  line=dict(color='red', width=2, dash='dot')),
        row=2, col=2
    )
    
    # Update layout
    fig.update_layout(
        height=800, 
        title_text=f"Analiză Completă Model LSTM - MAE: {metrics['mae']:.6f}, R²: {metrics['r2']:.4f}",
        showlegend=True
    )
    
    # Labels pentru axe
    fig.update_xaxes(title_text="Valori Reale", row=1, col=1)
    fig.update_yaxes(title_text="Predicții", row=1, col=1)
    fig.update_xaxes(title_text="Epocă", row=1, col=2)
    fig.update_yaxes(title_text="Loss", row=1, col=2)
    fig.update_xaxes(title_text="Reziduuri", row=2, col=1)
    fig.update_yaxes(title_text="Frecvență", row=2, col=1)
    fig.update_xaxes(title_text="Timp", row=2, col=2)
    fig.update_yaxes(title_text="Energie", row=2, col=2)
    
    fig.show()
    
    # 2. Plot separat pentru analiza FFT a predicțiilor
    create_fft_analysis_plot(y_test_orig, y_pred_orig)

def create_fft_analysis_plot(y_test_orig, y_pred_orig):
    """Analiză FFT a predicțiilor vs realitate"""
    
    # FFT pentru o subsecțiune reprezentativă
    sample_size = min(2048, len(y_test_orig))  # Putere de 2 pentru FFT eficient
    
    y_test_sample = y_test_orig[:sample_size]
    y_pred_sample = y_pred_orig[:sample_size]
    
    # Calculare FFT
    fft_test = np.abs(fft(y_test_sample))[:sample_size//2]
    fft_pred = np.abs(fft(y_pred_sample))[:sample_size//2]
    freqs = fftfreq(sample_size)[:sample_size//2]
    
    # Plot
    fig = go.Figure()
    
    fig.add_trace(
        go.Scatter(
            x=freqs, 
            y=fft_test, 
            name='FFT Original', 
            line=dict(color='blue', width=2)
        )
    )
    
    fig.add_trace(
        go.Scatter(
            x=freqs, 
            y=fft_pred, 
            name='FFT Predicții', 
            line=dict(color='red', width=2, dash='dash')
        )
    )
    
    # Evidențiază frecvența dominantă cunoscută (0.02 Hz)
    fig.add_vline(
        x=0.02, 
        line_dash="dot", 
        line_color="green",
        annotation_text="Freq dominantă (0.02 Hz)"
    )
    
    fig.update_layout(
        title="Comparație FFT: Original vs Predicții",
        xaxis_title="Frecvența (Hz)",
        yaxis_title="Amplitudine",
        height=500,
        showlegend=True
    )
    
    # Zoom pe regiunea de interes (0-0.1 Hz)
    fig.update_xaxes(range=[0, 0.1])
    
    fig.show()

# Rulare vizualizări
create_comprehensive_plots(results, history)

# %% [markdown]
# ## 11. Predicții pe Date Noi și Validare

# %%
def make_future_predictions(model, last_sequence, scalers, n_future_steps=100):
    """
    Creează predicții pentru viitor folosind ultimele date
    """
    print(f"🔮 Predicții pentru următorii {n_future_steps} pași...")
    
    # Folosește ultima secvență din datele de test
    current_sequence = last_sequence.copy()
    future_predictions = []
    
    # Predicție pas cu pas
    for step in range(n_future_steps):
        # Predicția pentru următorii OUT_STEPS
        pred = model.predict(current_sequence.reshape(1, current_sequence.shape[0], current_sequence.shape[1]), verbose=0)
        
        # Ia doar primul pas prezis
        next_energy = pred[0, 0]
        future_predictions.append(next_energy)
        
        # Actualizează secvența (remove primul element, add predicția)
        # Pentru simplitate, păstrăm doar energia (coloana 0) și aproximăm restul features
        new_row = current_sequence[-1].copy()
        new_row[0] = next_energy  # Update energia
        
        # Update secvența
        current_sequence = np.vstack([current_sequence[1:], new_row])
    
    # Scalare inversă pentru unități originale
    future_predictions = np.array(future_predictions).reshape(-1, 1)
    future_predictions_original = scalers['energy_scaler'].inverse_transform(future_predictions).flatten()
    
    return future_predictions_original

# Creează predicții pentru viitor
last_test_sequence = X_test[-1]  # Ultima secvență din test
future_pred = make_future_predictions(model, last_test_sequence, scalers, n_future_steps=100)

# Vizualizează predicțiile viitoare
fig = go.Figure()

# Ultimele valori cunoscute
known_values = scalers['energy_scaler'].inverse_transform(
    y_test[-10:].flatten().reshape(-1, 1)
).flatten()

x_known = np.arange(-len(known_values), 0)
x_future = np.arange(0, len(future_pred))

fig.add_trace(
    go.Scatter(
        x=x_known, 
        y=known_values, 
        name='Valori Cunoscute',
        line=dict(color='blue', width=3),
        mode='lines+markers'
    )
)

fig.add_trace(
    go.Scatter(
        x=x_future, 
        y=future_pred, 
        name='Predicții Viitoare',
        line=dict(color='red', width=2, dash='dash'),
        mode='lines+markers'
    )
)

fig.add_vline(x=0, line_dash="dot", line_color="black", annotation_text="Prezent")

fig.update_layout(
    title="Predicții pentru Viitor - Energie Cinetică",
    xaxis_title="Pași de Timp",
    yaxis_title="Energie Totală",
    height=500
)

fig.show()

print(f"📊 Statistici predicții viitoare:")
print(f"   ├── Min: {future_pred.min():.6f}")
print(f"   ├── Max: {future_pred.max():.6f}")
print(f"   ├── Mean: {future_pred.mean():.6f}")
print(f"   └── Std: {future_pred.std():.6f}")

# %% [markdown]
# ## 12. Analiza Importanței Features

# %%
def analyze_feature_importance_approximation(model, X_test, y_test, feature_names, n_samples=100):
    """
    Analiză aproximativă a importanței features prin permutare
    """
    print("🔍 Analiză importanță features (aproximativă)...")
    
    # Baseline performance
    baseline_pred = model.predict(X_test[:n_samples], verbose=0)
    baseline_mse = mean_squared_error(y_test[:n_samples].flatten(), baseline_pred.flatten())
    
    feature_importance = {}
    
    # Pentru fiecare feature
    for i, feature_name in enumerate(feature_names):
        print(f"   Testing feature: {feature_name}")
        
        # Creează o copie și amestecă feature-ul
        X_permuted = X_test[:n_samples].copy()
        X_permuted[:, :, i] = np.random.permutation(X_permuted[:, :, i].flatten()).reshape(X_permuted[:, :, i].shape)
        
        # Calculează performanța cu feature-ul amestecat
        permuted_pred = model.predict(X_permuted, verbose=0)
        permuted_mse = mean_squared_error(y_test[:n_samples].flatten(), permuted_pred.flatten())
        
        # Importanța = cât de mult se degradează performanța
        importance = permuted_mse - baseline_mse
        feature_importance[feature_name] = importance
    
    # Sortează și afișează
    sorted_importance = dict(sorted(feature_importance.items(), key=lambda x: x[1], reverse=True))
    
    print(f"\n🏆 Top 10 features importante:")
    for i, (feature, importance) in enumerate(list(sorted_importance.items())[:10]):
        print(f"   {i+1:2d}. {feature:20s}: {importance:.8f}")
    
    # Plot importanța
    fig = go.Figure()
    
    features = list(sorted_importance.keys())[:15]  # Top 15
    importances = [sorted_importance[f] for f in features]
    
    fig.add_trace(
        go.Bar(
            x=importances,
            y=features,
            orientation='h',
            marker=dict(color=importances, colorscale='Viridis')
        )
    )
    
    fig.update_layout(
        title="Importanța Features (Top 15)",
        xaxis_title="Degradarea MSE",
        yaxis_title="Features",
        height=600
    )
    
    fig.show()
    
    return sorted_importance

# Analiză importanță features
feature_importance = analyze_feature_importance_approximation(
    model, X_test, y_test, feature_names, n_samples=200
)

# %% [markdown]
# ## 13. Salvarea Modelului și Configurației

# %%
import json
import pickle
from datetime import datetime

def save_model_and_config(model, scalers, feature_names, results, config_info):
    """Salvează modelul și toate configurațiile"""
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    base_name = f"energy_lstm_model_{timestamp}"
    
    print(f"💾 Salvare model și configurație cu timestamp: {timestamp}")
    
    # 1. Salvare model TensorFlow
    model.save(f"{base_name}.h5")
    print(f"✅ Model salvat: {base_name}.h5")
    
    # 2. Salvare scalers
    with open(f"{base_name}_scalers.pkl", 'wb') as f:
        pickle.dump(scalers, f)
    print(f"✅ Scalers salvați: {base_name}_scalers.pkl")
    
    # 3. Salvare configurație și rezultate
    config = {
        'model_info': {
            'sequence_length': SEQUENCE_LENGTH,
            'output_steps': OUT_STEPS,
            'overlap_ratio': OVERLAP_RATIO,
            'total_features': len(feature_names),
            'training_samples': len(X_train),
            'test_samples': len(X_test)
        },
        'feature_names': feature_names,
        'performance_metrics': results['metrics'],
        'training_config': {
            'epochs_completed': len(history.history['loss']),
            'final_train_loss': float(history.history['loss'][-1]),
            'final_val_loss': float(history.history['val_loss'][-1]),
            'best_val_loss': float(min(history.history['val_loss']))
        },
        'data_info': config_info,
        'timestamp': timestamp
    }
    
    with open(f"{base_name}_config.json", 'w') as f:
        json.dump(config, f, indent=2)
    print(f"✅ Configurație salvată: {base_name}_config.json")
    
    return base_name

# Informații despre date pentru salvare
data_config = {
    'source_file': my_file,
    'total_data_points': len(df_data),
    'energy_range': [float(df_data['E_tot'].min()), float(df_data['E_tot'].max())],
    'temperature_range': [float(df_data['T'].min()), float(df_data['T'].max())],
    'dominant_frequency_hz': float(dominant_freqs[0]) if len(dominant_freqs) > 0 else None,
    'dominant_period_steps': float(1/dominant_freqs[0]) if len(dominant_freqs) > 0 and dominant_freqs[0] != 0 else None
}

# Salvare completă
saved_model_name = save_model_and_config(model, scalers, feature_names, results, data_config)

# %% [markdown]
# ## 14. Funcție pentru Încărcarea și Utilizarea Modelului Salvat

# %%
def load_trained_model(base_name):
    """Încarcă un model antrenat anterior"""
    print(f"📂 Încărcare model: {base_name}")
    
    try:
        # Încărcare model
        model = tf.keras.models.load_model(f"{base_name}.h5")
        print(f"✅ Model încărcat")
        
        # Încărcare scalers
        with open(f"{base_name}_scalers.pkl", 'rb') as f:
            scalers = pickle.load(f)
        print(f"✅ Scalers încărcați")
        
        # Încărcare configurație
        with open(f"{base_name}_config.json", 'r') as f:
            config = json.load(f)
        print(f"✅ Configurație încărcată")
        
        print(f"📊 Model info:")
        print(f"   ├── Features: {config['model_info']['total_features']}")
        print(f"   ├── Sequence length: {config['model_info']['sequence_length']}")
        print(f"   ├── Output steps: {config['model_info']['output_steps']}")
        print(f"   ├── Performance R²: {config['performance_metrics']['r2']:.4f}")
        print(f"   └── Performance MAE: {config['performance_metrics']['mae']:.6f}")
        
        return model, scalers, config
        
    except Exception as e:
        print(f"❌ Eroare la încărcare: {e}")
        return None, None, None

# Exemplu de utilizare (decomentează pentru a testa)
# loaded_model, loaded_scalers, loaded_config = load_trained_model(saved_model_name)

# %% [markdown]
# ## 15. Funcție pentru Predicții pe Date Noi

# %%
def predict_energy_sequence(model, scalers, feature_names, new_data, sequence_length):
    """
    Funcție pentru predicții pe date complet noi
    
    Args:
        model: Modelul antrenat
        scalers: Scalers pentru normalizare
        feature_names: Lista cu numele features
        new_data: DataFrame cu date noi (trebuie să aibă coloanele: E_tot, T, P)
        sequence_length: Lungimea secvenței de input
    """
    print(f"🔮 Predicții pe {len(new_data)} puncte noi...")
    
    try:
        # Verifică că datele au coloanele necesare
        required_cols = ['E_tot', 'T', 'P']
        if not all(col in new_data.columns for col in required_cols):
            raise ValueError(f"Datele trebuie să conțină coloanele: {required_cols}")
        
        # Recreate features similar cu training
        energy_scaled = scalers['energy_scaler'].transform(new_data[['E_tot']]).flatten()
        temp_scaled = scalers['temp_scaler'].transform(new_data[['T']]).flatten()
        pressure_scaled = scalers['pressure_scaler'].transform(new_data[['P']]).flatten()
        
        # Features derivate
        energy_diff = np.gradient(energy_scaled)
        energy_diff2 = np.gradient(energy_diff)
        
        energy_ma5 = pd.Series(energy_scaled).rolling(window=5, center=True).mean().fillna(method='bfill').fillna(method='ffill')
        energy_ma10 = pd.Series(energy_scaled).rolling(window=10, center=True).mean().fillna(method='bfill').fillna(method='ffill')
        energy_ma20 = pd.Series(energy_scaled).rolling(window=20, center=True).mean().fillna(method='bfill').fillna(method='ffill')
        
        energy_vol5 = pd.Series(energy_scaled).rolling(window=5, center=True).std().fillna(0)
        energy_vol10 = pd.Series(energy_scaled).rolling(window=10, center=True).std().fillna(0)
        
        temp_diff = np.gradient(temp_scaled)
        temp_ma10 = pd.Series(temp_scaled).rolling(window=10, center=True).mean().fillna(method='bfill').fillna(method='ffill')
        
        # Features Fourier (aproximare - ar trebui să fie calculate similar cu training)
        fourier_features, _ = create_fourier_features(energy_scaled, dominant_freqs, top_n=3)
        
        # Combine toate features
        all_features = np.column_stack([
            energy_scaled, temp_scaled, pressure_scaled,
            energy_diff, energy_diff2,
            energy_ma5, energy_ma10, energy_ma20,
            energy_vol5, energy_vol10,
            temp_diff, temp_ma10,
            fourier_features
        ])
        
        predictions = []
        
        # Predicții pentru fiecare secvență posibilă
        for i in range(len(all_features) - sequence_length + 1):
            seq = all_features[i:i + sequence_length].reshape(1, sequence_length, -1)
            pred = model.predict(seq, verbose=0)
            predictions.append(pred[0])
        
        # Scalare inversă
        predictions = np.array(predictions)
        predictions_original = scalers['energy_scaler'].inverse_transform(
            predictions.reshape(-1, 1)
        ).reshape(predictions.shape)
        
        print(f"✅ {len(predictions)} predicții generate")
        
        return predictions_original
        
    except Exception as e:
        print(f"❌ Eroare la predicție: {e}")
        return None

# %% [markdown]
# ## 16. Rezumat Final și Recomandări

# %%
print("🎯 REZUMAT FINAL - ANALIZA TIME SERIES ENERGIE CINETICĂ")
print("="*60)

print(f"\n📊 PERFORMANȚA MODELULUI:")
print(f"   ├── R² Score: {results['metrics']['r2']:.6f}")
print(f"   ├── Mean Absolute Error: {results['metrics']['mae']:.8f}")
print(f"   ├── Root Mean Square Error: {results['metrics']['rmse']:.8f}")
print(f"   └── Mean Absolute Percentage Error: {results['metrics']['mape']:.4f}%")

print(f"\n🔧 CONFIGURAȚIA OPTIMALĂ:")
print(f"   ├── Sequence Length: {SEQUENCE_LENGTH} pași (2× perioada principală)")
print(f"   ├── Output Steps: {OUT_STEPS} pași (0.5× perioada)")
print(f"   ├── Total Features: {len(feature_names)}")
print(f"   ├── Overlap Ratio: {OVERLAP_RATIO} ({int(OVERLAP_RATIO*100)}%)")
print(f"   └── Training Samples: {len(X_train):,}")

print(f"\n🌊 ANALIZA FOURIER:")
if len(dominant_freqs) > 0:
    print(f"   ├── Frecvența dominantă: {dominant_freqs[0]:.6f} Hz")
    print(f"   ├── Perioada principală: {1/dominant_freqs[0]:.2f} pași")
    print(f"   └── Features Fourier generate: {len([f for f in feature_names if 'sin_' in f or 'cos_' in f])}")

print(f"\n💡 RECOMANDĂRI PENTRU OPTIMIZARE ULTERIOARĂ:")
print(f"   ├── Experimentează cu arhitecturi Transformer pentru capturarea dependințelor")
print(f"   ├── Încearcă ensemble methods cu multiple modele LSTM")
print(f"   ├── Consideră attention mechanisms pentru features importante")
print(f"   ├── Testează data augmentation prin rotații de fază")
print(f"   └── Explorează autoencoder pentru detecția anomaliilor")

print(f"\n📁 FIȘIERE SALVATE:")
print(f"   ├── Model: {saved_model_name}.h5")
print(f"   ├── Scalers: {saved_model_name}_scalers.pkl")
print(f"   └── Config: {saved_model_name}_config.json")

print(f"\n🚀 URMĂTORII PAȘI:")
print(f"   1. Testează modelul pe date complet noi")
print(f"   2. Implementează monitoring în timp real")
print(f"   3. Optimizează hiperparametrii cu Optuna/Hyperopt")
print(f"   4. Dezvoltă API pentru predicții în producție")

print("\n✅ ANALIZA COMPLETĂ FINALIZATĂ!")
print("="*60)

2025-08-11 11:21:25.712190: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-11 11:21:25.721650: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-08-11 11:21:25.733406: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-08-11 11:21:25.736717: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-08-11 11:21:25.746660: I tensorflow/core/platform/cpu_feature_guar

📦 Libraries importate cu succes!
📊 Dimensiune date originale: (93300, 6)
📋 Coloane disponibile: ['Step', 'T', 'E_KS', 'E_tot', 'Vol', 'P']
🔢 Numărul de steps unici: 100
✅ Dimensiunea după filtrare: 90000 rânduri
📈 Range energie totală: [-2131.327240, -2130.745750]
🌡️ Range temperatură: [1.4500, 53.0400]

🔍 Valori lipsă per coloană:
Step     0
T        0
E_KS     0
E_tot    0
Vol      0
P        0
dtype: int64
🔍 Analiză periodicitate cu FFT...
🎯 Top 10 frecvențe dominante:
    1. Freq: 0.020000 Hz, Perioadă:    50.00 pași, Putere: 7.49e+03
    2. Freq: 0.021111 Hz, Perioadă:    47.37 pași, Putere: 4.20e+03
    3. Freq: 0.001111 Hz, Perioadă:   900.00 pași, Putere: 3.00e+03
    4. Freq: 0.018889 Hz, Perioadă:    52.94 pași, Putere: 2.06e+03
    5. Freq: 0.041111 Hz, Perioadă:    24.32 pași, Putere: 1.78e+03
    6. Freq: 0.002222 Hz, Perioadă:   450.00 pași, Putere: 1.54e+03
    7. Freq: 0.022222 Hz, Perioadă:    45.00 pași, Putere: 1.53e+03
    8. Freq: 0.017778 Hz, Perioadă:    56.25 pa

🌊 Creare 3 features Fourier...
✅ Features Fourier create: ['sin_f1_0.0200Hz', 'cos_f1_0.0200Hz', 'sin_f2_0.0211Hz', 'cos_f2_0.0211Hz', 'sin_f3_0.0011Hz', 'cos_f3_0.0011Hz']
📏 Shape features Fourier: (90000, 6)
🔧 Creare features comprehensive...
✅ Total features: 18
📋 Feature names: ['Energy_scaled', 'Temp_scaled', 'Pressure_scaled', 'Energy_velocity', 'Energy_acceleration', 'Energy_MA5', 'Energy_MA10', 'Energy_MA20', 'Energy_Vol5', 'Energy_Vol10', 'Temp_velocity', 'Temp_MA10', 'sin_f1_0.0200Hz', 'cos_f1_0.0200Hz', 'sin_f2_0.0211Hz', 'cos_f2_0.0211Hz', 'sin_f3_0.0011Hz', 'cos_f3_0.0011Hz']

🔍 Verificare calitatea features:
NaN values: 0
Inf values: 0
Feature range: [-4.3568, 3.8736]
🔧 Creare secvențe cu parametri:
   - Sequence length: 100
   - Output steps: 25
   - Overlap ratio: 0.7
✅ Secvențe create:
   - Input shape: (2996, 100, 18)
   - Target shape: (2996, 25)
   - Total samples: 2996
🏗️ Construire model LSTM pentru:
   - Input shape: (100, 18)
   - Output steps: 25
   - Dropout r

2025-08-11 11:21:28.352487: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2025-08-11 11:21:28.352733: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:135] retrieving CUDA diagnostic information for host: hondar
2025-08-11 11:21:28.352745: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:142] hostname: hondar
2025-08-11 11:21:28.352881: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:166] libcuda reported version is: 535.247.1
2025-08-11 11:21:28.352897: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:170] kernel reported version is: 535.247.1
2025-08-11 11:21:28.352901: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:249] kernel version seems to match DSO: 535.247.1


Model: "sequential"

...

 Trainable params: 341,465 (1.30 MB)

 Non-trainable params: 832 (3.25 KB)

📊 Împărțirea datelor:
   - Train samples: 2396
   - Test samples: 600
   - Features per sample: 18
   - Sequence length: 100
🚀 Începe antrenarea...
Epoch 1/100
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step - loss: 0.4721 - mae: 0.8749 - mape: 152.0317 - mse: 1.0656
Epoch 1: val_loss improved from inf to 0.42202, saving model to best_energy_lstm_model.h5


75/75 ━━━━━━━━━━━━━━━━━━━━ 17s 122ms/step - loss: 0.4718 - mae: 0.8746 - mape: 151.8707 - mse: 1.0649 - val_loss: 0.4220 - val_mae: 0.8273 - val_mape: 102.9648 - val_mse: 0.9163 - learning_rate: 0.0010
Epoch 2/100
64/75 ━━━━━━━━━━━━━━━━━━━━ 1s 106ms/step - loss: 0.3850 - mae: 0.7639 - mape: 132.0340 - mse: 0.8471

KeyboardInterrupt: 